<a href="https://colab.research.google.com/github/OlegV12/NaturalLangProcessing/blob/Lesson_9/HW_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import pandas as pd
import torch
from torch import nn
import torch
import pandas as pd
from collections import Counter
from torch.autograd import Variable

In [3]:
torch.cuda.is_available()

True

In [4]:
text = open('/gdrive/MyDrive/Colab Notebooks/NLP/evgenyi_onegin.txt', 'rb').read().decode(encoding='utf-8')

In [5]:
vocab = sorted(set(text))

In [ ]:
text.split()

In [7]:
test_sentence = text.split()

In [8]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
chunk_len=len(trigrams)
print(trigrams[:3])

[(['Александр', 'Сергеевич'], 'Пушкин'), (['Сергеевич', 'Пушкин'], 'Евгений'), (['Пушкин', 'Евгений'], 'Онегин')]


In [9]:
vocab = set(test_sentence)
voc_len=len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [10]:
voc_len

11512

In [11]:
inp=[]
tar=[]
for context, target in trigrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        inp.append(context_idxs)
        targ = torch.tensor([word_to_ix[target]], dtype=torch.long)
        tar.append(targ)

In [12]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size*2, hidden_size, n_layers,batch_first=True,
                          bidirectional=False)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [14]:
def train(inp, target):
    hidden = decoder.init_hidden().cuda()
    decoder.zero_grad()
    loss = 0
    
    for c in range(chunk_len):
        output, hidden = decoder(inp[c].cuda(), hidden)
        loss += criterion(output, target[c].cuda())

    loss.backward()
    decoder_optimizer.step()

    return loss.data.item() / chunk_len

In [ ]:
n_epochs = 50
print_every = 10
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.015

decoder = RNN(voc_len, hidden_size, voc_len, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0
decoder.cuda()
for epoch in range(n_epochs):
    loss = train(inp, tar)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 50, loss))


    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 22s (0 0%) 9.3843]
 8s (10 10%) 4.2287]
 54s (20 20%) 1.0310]
 40s (30 30%) 0.2352]


In [ ]:
def evaluate(prime_str, predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden().cuda()

    for p in range(predict_len):
        
        prime_input = torch.tensor([word_to_ix[w] for w in prime_str.split()], dtype=torch.long).cuda()
        inp = prime_input[-2:] #last two words as input
        output, hidden = decoder(inp, hidden)

        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted word to string and use as next input
        predicted_word = list(word_to_ix.keys())[list(word_to_ix.values()).index(top_i)]
        prime_str += " " + predicted_word

    return prime_str

In [19]:
print(evaluate('он думал', 40, temperature=1))

он думал да молвил от утренних долин В ученым прямо и теплотой Камин чуть дышит. Дым из трубок В трубу уходит. Светлый кубок Еще шипит среди стола. Вечерняя находит мгла... (Люблю я дружеские враки И дружеский бокал вина Порою той, что названа


In [20]:
print(evaluate('он думал', 40, temperature=0.5))

он думал бродит, Меня смотрит был мой Онегин поскакал. Перед померкшими домами Вдоль сонной улицы рядами Двойные фонари карет Веселый изливают свет И радуги на снег наводят; Усеян плошками кругом, Блестит великолепный дом; По цельным окнам тени ходят, Мелькают профили голов И
